# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [16]:
!pip install -Uq openai-agents

In [17]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [18]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [21]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-2.0-flash-lite-preview-02-05:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [27]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1742929809-Y1kGIWKn14EWRZY5usb1', 'provider': 'Google', 'model': 'google/gemini-2.0-flash-lite-preview-02-05', 'object': 'chat.completion', 'created': 1742929809, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': 'The meaning of life is a question that has plagued philosophers, theologians, and individuals for centuries. There\'s no single, universally accepted answer, and the beauty lies in the exploration and the personal meaning we create. Here\'s a breakdown of different perspectives:\n\n**1. Philosophical Approaches:**\n\n*   **Nihilism:** This philosophy argues that life is inherently meaningless, without objective purpose, value, or truth. There\'s nothing to strive for beyond the immediate.\n*   **Existentialism:** Emphasizes individual freedom and responsibility. We are born into the world without a predetermined purpose, and it\'s up to us to create our own meaning throug

In [29]:
data = response.json()
data['choices'][0]['message']['content']

'The meaning of life is a question that has plagued philosophers, theologians, and individuals for centuries. There\'s no single, universally accepted answer, and the beauty lies in the exploration and the personal meaning we create. Here\'s a breakdown of different perspectives:\n\n**1. Philosophical Approaches:**\n\n*   **Nihilism:** This philosophy argues that life is inherently meaningless, without objective purpose, value, or truth. There\'s nothing to strive for beyond the immediate.\n*   **Existentialism:** Emphasizes individual freedom and responsibility. We are born into the world without a predetermined purpose, and it\'s up to us to create our own meaning through our choices and actions. Key figures include Jean-Paul Sartre and Albert Camus.\n*   **Absurdism:** Acknowledges the conflict between the human desire for meaning and the seemingly meaningless universe. Embraces the absurdity of existence, and encourages a defiant embrace of life despite its inherent lack of purpose

## 2. Using OpenAI Agents SDK

In [22]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

A function calls self,
Breaking tasks into smaller,
Until base is reached.

